**DO NOT EDIT THIS FILE WITHIN THE /TSDS FOLDER - YOU RISK OVERWRITING YOUR WORK THE NEXT TIME YOU PULL FROM THE GITHUB REPOSITORY**

# Assignment 3


### Practical info
* Handin in absalon. The deadline is the 23rd of April (see the [course plan](https://github.com/abjer/tsds/wiki/Course-plan))
* You must work in groups of 2-4. **Remember to identify the group members in the filename or in the top of the file contents**.
* If anything is unclear dont hesitate to email me at kuol@econ.ku.dk with questions.

<br>

<br>
<br>
<br>
<br>

# Questions from exercise set 9 (Spatial #1)


In [ ]:
import os
import requests

import numpy as np
import pandas as pd
import seaborn as sns

import folium
import geopandas as gpd
import fiona
import shapely

%matplotlib inline

In [ ]:
# Run this code to get set up for exercise 9.2.2 - 9.2.5

url = "https://raw.githubusercontent.com/ok-dk/dagi/master/geojson/kommuner.geojson"
kommuner = gpd.read_file(url)
print(kommuner.crs)
kommuner.to_crs('epsg:25832', inplace=True)
kommuner.plot(figsize=(14,8))

kommuner.columns = kommuner.columns.str.lower()
    
# information for municipality
kommune_info = pd.read_json('https://dawa.aws.dk/kommuner')\
                .pipe(lambda df: \
                        df.assign(komkode=df.kode.astype(str).str.zfill(4)))\
                .loc[:,['komkode','regionskode']]\
                

region_info = pd.read_json('https://dawa.aws.dk/regioner/')\
                .loc[:,['kode','navn']]\
                .add_prefix('regions')

kommuner = kommuner\
                .merge(kommune_info,how='left')\
                .merge(region_info,how='left')


dk_crs = {'ellps': 'GRS80', 'no_defs': True, 'proj': 'utm', 'units': 'm', 'zone': 32}
def cell_coords_to_polygons(square_df, x='e', y='n', dist=500, crs=dk_crs):
    '''
    Convert coordinates to squares in a GeoDataFrame.
       
    Parameters
    ----------
    x : str
        Name of the horizontal coordinate (~longitude)            
    y : str
        Name of the vertical coordinate (~latitude)                        
    dist : int or float
        Size of polygons
    crs : dict
        Coordinate Reference System


    Returns
    ----------
    squares_gdf: geopandas.GeoDataFrame
        This table contains squares as geometry
        and the original data.
    '''
    
    def _to_square_polygon(row):
        '''
        This auxiliary function convert a square's lower,left 
        coordinates to a polygon. 
        
        Parameters
        ----------
        row : pandas.Series
            This is a DataFrame row.            
        
        Returns
        ----------
        poly: shapely.Polygon        
        
        '''
        
        square_coords = ((row[x], row[y]), 
                         (row[x]+dist, row[y]), 
                         (row[x]+dist, row[y]+dist), 
                         (row[x], row[y]+dist))
        
        poly = shapely.geometry.Polygon(square_coords)
        
        return poly
    
    # convert to polygons
    square_geoms = gpd.GeoSeries(square_df.apply(_to_square_polygon, axis=1), crs=crs)
    
    # make GeoDataFrame
    square_gdf = gpd.GeoDataFrame(data=square_df, geometry=square_geoms)
    
    return square_gdf

**Ex. 9.2.2** Make two boolean series as follows. 
 - first: row is True if corresponding the row shape is in the Capital Region or Sealand Region (i.e. `'Region Hovedstaden', 'Region Sjælland'`) 
 - second: row is True if the  the row geometry is ***not*** in Bornholm or nearby (i.e. `'Bornholm', 'Christiansø'`)

Finally make a combined series which takes the value True if both holds, otherwise False and use this series to select rows in the GeoDataFrame. 

> *Hint*: recall that we can check if a series elements are elements in a series using the `isin` method


In [ ]:
# [Answer to ex. 9.2.2 here]

**Ex. 9.2.3** Explain what is the CRS of the GeoDataFrame. How is distance measured using this coordinate system. Extract the extremum values (min,max) in all dimensions.

> *Hint*: extreme values, i.e. bounds, can be found using `.bounds` on a GeoDataFrame (also works on shapes, GeoSeries)

In [ ]:
# [Answer to ex. 9.2.3 here]

#### Interpolation of house prices


In the following two exercises we aim to compute local spatial neighborhood measures of house prices in Sealand and around. We do this by making make a grid of Sealand houseprice data. Thus the exercise will illustrate how to make a interpolation of data which is useful for feature engineering and get a good understanding of the data.

**Ex. 9.2.4** We are now to construct a 500mx500m grid for Sealand:
- Make a grid of points 500m apart in horizontal and vertical directions that are within the extremum values of Sealand's shape. 
- For each of these points construct a square polygon assuming that the point is the south west corner of the square. 
- Select all the house sales that take place within the Sealand and nearby islands.

> *Hint 1:* Once you have created the grid the following function below may be useful for converting into a GeoDataFrame. You need to specify the column names for your x and y coordinates.

> *Hint 2:* We can select the points that intersect by using a spatial join between the house locations and municipalities.

In [ ]:
# [Answer to ex. 9.2.4 here]

**Ex. 9.2.5** Compute interpolation of house price for each grid cell as follows:
- Load the pre-structured data with house sales prices for the capital region of Denmark
- Make a loop over sale_year
    - Fit a nearest neighbor regression model to the square meter price (i.e. `price_area` for each year)
        - Set number of neighbors to 25 and radius to 25000
    - Apply the model to the grid data and assign as a column
- Plot the grid data for 2012

In [ ]:
# [Answer to ex. 9.2.5 here]

<br>
<br>
<br>
<br>

# Questions from exercise set 10 (Spatial #2)

In [ ]:
import shapely as shp
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# You need to download these files to get going. See the exercise set 
# for instructions.
districts = gpd.read_file('data/school_districts_2017_2018.geojson')
socioeco = gpd.read_file('data/socioeco_edited.geojson')
school_perf = pd.read_csv('data/school_perf.csv')
house_sales = gpd.read_file('data/sales.geojson')

cph_map = gpd.read_file('data/cph_map.geojson').iloc[0,0]

districts = districts[['skoleid','skolenavn','geometry']]
house_sales = house_sales[house_sales['year']>2016]

percs = house_sales['price_m2'].quantile([0.01,0.99])

sales_dist_plt = plt.subplots(1,2,sharey=True,figsize=(12,4))
sns.distplot(house_sales['price_m2'], ax=sales_dist_plt[1][0]) 
house_sales = house_sales[(percs[0.01]<house_sales['price_m2']) & (house_sales['price_m2']<percs[0.99])]
sns.distplot(house_sales['price_m2'], ax=sales_dist_plt[1][1])
sales_dist_plt[1][0].set_title('With outliers')
sales_dist_plt[1][1].set_title('Without outliers')
sales_dist_plt[0].suptitle('Distribution of square meter prices')
plt.show()

districts_perf = districts.merge(school_perf[school_perf['year']<2017].drop('year',axis=1).groupby('skolenavn',as_index=False).mean())

socioeco_sub = socioeco\
    .loc[socioeco['aar']==2016,['rode_nr','geometry']]\
    .merge(socioeco[socioeco['aar']>2012]\
               .groupby('rode_nr',as_index=False)\
               .mean()
          )\
    .drop('aar',axis=1)


districts_perf_plt = plt.subplots(1,2,figsize=(18,8), gridspec_kw = {'width_ratios':[1,1.25]})
districts.plot(color='lightgrey',edgecolor='black',ax=districts_perf_plt[1][0])
districts.plot(color='lightgrey',edgecolor='black',ax=districts_perf_plt[1][1])
districts_perf.plot(edgecolor='black',column='karsnit',legend=True,ax=districts_perf_plt[1][1])
house_sales.plot(color='red', markersize=1,ax=districts_perf_plt[1][1])
districts_perf_plt[1][0].set_axis_off()
districts_perf_plt[1][1].set_axis_off()
plt.show()

**Ex 10.1.1**: *Border regions between school districts*.
 Make a function that finds the borders between the school districts given some specified border-width W. Make sure that your border building function satisfy the following constraints:
 - The borders should not intersect each other. There should be no borders between the school districts without any grade data and any other school districts. 
 - Do not include borders between two school districts, if the district border coincides with a natural geographical border, such that the neighborhoods on each side of the geographical border are likely to be systematically different with regard to other variables than the school characteristics. For instance, it would be natural to exclude the Christianhavn school district from the analysis, since it is surrounded by the harbor and *Voldene*, and likewise it would be natural not to include the border between two school districts, if they are on different sides of the harbor. 

 Compute the borders for the following border meter widths: 50, 100, 200, 300, 500. A border width of, for instance, 50 meters means that the computed border area should reach 25 meter into each district along the actual border:

> **Hint**: One approach is to use a combination of the *buffer* and *overlay* function from *Geopandas* to construct the borders. Depending on your choice of method, the found borders will likely overlap, but you can then use *overlay* to find the overlap. After you have found the overlaps, you can use the *shapely* function *difference* to remove them. You can use a similar approach with the *cph_map* from above to remove borders between school districts on different sides of the harbor.

> **Hint 2**: The borders produced should look like this.

<img src="https://github.com/abjer/tsds/raw/master/material/10_spatial2/border_plt.png">


In [ ]:
# [Answer to ex. 10.1.1 here]

**Ex 10.1.2**: Do the following
  1. Merge the residential sales with the school districts and socioeconomic data based on the coordinates of the sale and the school districts and rode areas (CPH municapality's spatial cells). 
  2. Find the residential sales located within the sets of borders for the different border widths. 
  3. Plot the located within the border sets for the different border widths.

> **Hint**: Recall the `sjoin` function from `geopandas` may be of help.

The selected houses within borders should look like:

<img src="https://github.com/abjer/tsds/raw/master/material/10_spatial2/border_sales_plt.png">


In [ ]:
# [Answer to ex. 10.1.2 here]

<br>

**Ex 10.1.3**:  Run a linear regression of the effect of schools' grade average on square meter prices, where you control for the size of the sold property and relevant socioeconomic properties of the neighborhood. We have used percent of non-western immigrants and descendants, low income, non-educated and high school educated as our socioeconomic variables, but you can play around with it.

In [ ]:
# [Answer to ex. 10.1.3 here]

<br>

**Ex 10.1.4**:  For each border width now try to run a linear regression of the effect of schools' average grade on square meter prices, where you only use the sales within the borders and control for residential area, socioeconomic neighborhood properties and border fixed effects.

In [ ]:
# [Answer to ex. 10.1.4 here]

<br>

**Ex 10.1.5**:  Compare the estimated effect from the controlled linear regression on all sales with the estimated effects from the linear regressions on the different border sets. The figure below show a summary of the results, we get from our analysis. The blue lines are confidence intervals of the effect of schools' average grade on house prices estimated with the different border sets (we do not use clustering). The grey area shows the confidence interval from the controlled linear regression model on all the sales.

<img src="https://github.com/abjer/tsds/raw/master/material/10_spatial2/results_fig.png">

 Would you say that the results suggest that there is a causal effect of the local shool's average grade on square meter price? Would you have expected that the estimated effect declines, when borders of greater width are used to estimate it? If not, what could explain that we this? Investigate your hypothesis, if you have time  for it (and tell us, if you find something interesting).


In [ ]:
# [Answer to ex. 10.1.5 here]

*Answer:* We see that estimates using border fixed effects are all below the confidence interval of the cross sectional analysis. This indicates that the cross-section analysis is biased. We see that for larger border the effect vanishes - see next questions for possible explanations.